In [1]:
from pyspark import SparkContext , SparkConf
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
from scipy.sparse import *

In [2]:
conf = SparkConf().setAppName("App")
conf.set("spark.driver.memory", '4G')
#conf = (conf.setMaster('local[*]')
        #.set('spark.driver.memory', '4G')
sc = SparkContext(conf=conf)

In [3]:
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), float(line[2])))

In [4]:
relevantsPerUser = trainSet.filter(lambda line: line[2] >= 8)\
                            .map(lambda line: (line[0], [line[1]]))\
                            .reduceByKey(lambda x, y: x + y)

In [5]:
relevantsDict = relevantsPerUser.collectAsMap()

In [6]:
import numpy as np

def ap(RankedList, PositiveItems, at=None):
    """
    Calculates AP@_ 
    """
    print("Recomended: ", RankedList)
    RankedList = RankedList[:at]
    is_relevant = np.in1d(RankedList, PositiveItems, assume_unique=True)
    print("Positives: ", PositiveItems)
    print("is_relevant: ", is_relevant)   
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(len(is_relevant)))
    print("P at k :" , p_at_k)
    map_score = np.sum(p_at_k) / np.min([len(PositiveItems), len(RankedList)])
    print(map_score)
    assert 0 <= map_score <= 1, map_score
    
    return map_score

In [7]:
icm = sc.textFile("icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
#FOR THE TOP POP
itemSet = trainSet.map(lambda x: (x[1], x[2]))
itemsCount = trainSet.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x + y)
itemsCount_dict = itemsCount.collectAsMap()
#-----------------------------------------
featureFreq = icm.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
featureFreqDict = featureFreq.collectAsMap()
prodCount= icm.map(lambda x: x[0]).distinct().count()
featureIdf = featureFreq.map(lambda x: (x[0],np.log10(prodCount/x[1])))
featureIdfDict = featureIdf.collectAsMap()

targetUsers = sc.textFile("target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

######
#TEST COMPUTING THE PREDICTION NORMALIZING BY THE N OF FEATURES AND NOT THE SQRT OF IT
norms = icm.map(lambda x: (x[0],1))\
                .reduceByKey(lambda x, y: x+y).mapValues(lambda x: np.sqrt(x))\
                .collectAsMap()

normalized = icm.map(lambda x: (x[0], x[1], x[2]/norms[x[0]]))

In [8]:
len(targets)

4196

In [9]:
#Just consider the row of the users to predict
#IF you0re asking, only the idf depends on all the training set, in fact it is computed before reducing the trainset
trainSet=trainSet.filter(lambda x: x[0] in targets)
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
data.append(0)
rows.append(15364)
cols.append(37142)
userItem=csr_matrix((data,(rows,cols)))
data = normalized.map(lambda x: x[2]).collect()
rows = normalized.map(lambda x: x[0]).collect()
cols = normalized.map(lambda x: x[1]).collect()
data.append(0)
rows.append(37142)
cols.append(80)
itemFeature = csc_matrix((data,(rows,cols)))

In [10]:
userFeature = userItem.dot(itemFeature)
userFeature.shape

(15365, 19716)

In [11]:
data = []
rows = []
cols = []
for f in featureIdfDict.keys():
    data.append(featureIdfDict[f])
    cols.append(f)
    rows.append(f)
featureIdf = csr_matrix((data,(rows,cols)))
featureIdf.shape

(19716, 19716)

In [12]:
userProfile = userFeature.dot(featureIdf)
prediction = userProfile.dot(itemFeature.transpose())
prediction.shape

(15365, 37143)

In [13]:
numberOfRecommendations=5
#TOP POPULAR
cost=8
avgRatings=itemSet.reduceByKey(lambda x,y: x+y)
avgRatings=avgRatings.map(lambda x: (x[0],x[1]/(itemsCount_dict[x[0]]+cost)))
avgRatings.take(5)
itemOrderByPop=avgRatings.sortBy(lambda x: x[1], ascending=False)
itemPop = np.array(itemOrderByPop.map(lambda x: x[0]).collect())
seenItems= trainSet.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x + y)
seenItemsDict=seenItems.collectAsMap()
#--------------------------------------------------------------

def recommendTopPop(user_id, removeSeen=True):
    seenItems = np.array(seenItemsDict[user_id])
    recommendedList = itemPop
    if(removeSeen):
        unseen_mask = np.in1d(recommendedList, seenItems, invert=True)
        recommendedList = recommendedList[unseen_mask]       
    return recommendedList[:numberOfRecommendations]


def fillWithTopPop(recommended,user):
    TopPop=recommendTopPop(user)
    for i in range (numberOfRecommendations-len(recommended)):
        recommended.append(TopPop[i])
    return recommended


def getRecommended(user):
    recommended = []
    itemsPred = prediction.getrow(user).toarray()[0]

    for i in range(0,len(itemsPred)):
        if(itemsPred[i]!=0):
                #if i not in seenItemsDict[user]:
                recommended.append((i, itemsPred[i]))
    recommended.sort(key = lambda x: -x[1])
    recommended=recommended[:numberOfRecommendations]
    print(recommended)
    recommendedItems = list(map(lambda x: x[0], recommended))
    if(len(recommendedItems)<numberOfRecommendations):
        recommendedItems=fillWithTopPop(recommendedItems, user)
    return recommendedItems    

In [14]:
recomendations = sc.parallelize(targets)

In [19]:
recomendations = recomendations.map(lambda x: (x, getRecommended(x)))

In [ ]:
recomendations.take(5)

In [ ]:
recomendations.map(lambda x: (x[0], ap(x[1],relevantsDict[x[0]])))